In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os
from requests_html import HTMLSession
session = HTMLSession()

In [ ]:
def get_article(url):
    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")


    article_tag = soup.find("div", {"class": "sinarharian-article-details"}).find("article")

    if article_tag:
        article = article_tag

        for div in article.find_all("div", {"class": "related-article-inside-body"}):
            div.decompose()
    else:
        article = None

    author_name = soup.find("span", {"class": "author"}).text.strip() if soup.find("span", {"class": "author"}) else None
    reading_time = (
        soup.find("span", {"class": "readingTime"}).find("span", {"id": "readingtime"}).text
    )
    article_dt = soup.find("span", {"class": "timespan"}).text if soup.find("span", {"class": "timespan"}) else None
    article_url = soup.find("div", {"id": "article_info"}).get("data-article_url") if soup.find("div", {"id": "article_info"}) else None
    article_title = soup.find("div", {"id": "article_info"}).get("data-article_title") if soup.find("div", {"id": "article_info"}).get("data-article_title") else None

    if article:
        article_text = article.text.strip()
    else: 
        article_text = None

    data = {
        "author_name": author_name,
        "article_url": article_url,
        "article_title": article_title,
        "article_text": article_text,
        "reading_time": reading_time,
        "article_dt": article_dt,
    }
    
    return data


In [ ]:
def write_to_json(lst, fn):
    import json
    with open(fn, "w+", encoding="utf-8") as file:
        for item in lst:
            x = json.dumps(item)
            file.write(x + "\n")

In [ ]:
param = "politik"
fn = f"output/sinar_harian_link_{param}.txt"
fn_file = fn.split("/")[1]

with open(fn, "r") as link:
    lines = link.readlines()
    lines = [item.strip() for item in lines]
    
result_list = []

for item in lines:
    try:
        data = get_article(item)
        result_list.append(data)
    except TimeoutError:
        import time
        time.sleep(10)
        continue
    except:
        time.sleep(60)
            
    
    write_to_json(result_list, f"output/sinar_harian_link_{param}.json")

In [ ]:
result_list